In [17]:
from textwrap import wrap
def convert_int(x):
    return int(x,2)

In [15]:
vector_data = sc.textFile('../data-fingerprint/2002_DB1a_data/Vector_Index/1_1.tif.bmp.wsq.txt')

In [18]:
vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).first()

['00000011',
 '11110000',
 '00000011',
 '11111111',
 '00000001',
 '11111111',
 '11100000',
 '11111111',
 '11111100',
 '01111111',
 '11111111',
 '10011111',
 '11111111',
 '11101111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '11111111',
 '01111111',
 '11111111',
 '10011111',
 '11111111',
 '11100011',
 '11111111',
 '11110000',
 '01111111',
 '11111000',
 '00001111',
 '11111100',
 '00000000',
 '11111100',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',

In [19]:
vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).map(lambda x: list(map(lambda x1: int(x1,2),x))).first()

[3,
 240,
 3,
 255,
 1,
 255,
 224,
 255,
 252,
 127,
 255,
 159,
 255,
 239,
 255,
 255,
 255,
 255,
 255,
 255,
 255,
 255,
 255,
 255,
 255,
 255,
 255,
 127,
 255,
 159,
 255,
 227,
 255,
 240,
 127,
 248,
 15,
 252,
 0,
 252,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 16,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 99,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 24,
 194,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 48,
 128,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 140,
 32,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 33,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 16,
 132,
 32,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 115,
 136,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 28,
 226,
 0,
 17,
 136,
 0,
 0,
 0,
 0,
 0,
 0,
 2

### Prepocessing( Create Index)

In [5]:
hashs = sc.parallelize(list(range(192)))

In [11]:
def create_index(pos):
    from elasticsearch import Elasticsearch
    es = Elasticsearch()
    request_body = {
        "settings" : {
            "number_of_shards": 5,
            "number_of_replicas": 1
        },
        "mappings": {
            "pos": {
                "properties": {
                    "text": {"type":"text"}
                }
            }
        }
    }


    es.indices.create(index = 'hash{}'.format(pos), body = request_body,request_timeout=10000)
    return 1


hashs.map(lambda x:create_index(x)).collect()


creating 'Hash1 ' index...


### Inserir no Índice

In [60]:
from elasticsearch import helpers
def gendata(list_pos_in_hash, id):
    for i,pos in enumerate(list_pos_in_hash):
        if(pos != 0):
            yield {
                    "_index": "hash{}".format(i),
                    "_type": "pos",
                    "_id": str(pos),
                    "_source": {
                    "text": "P{}_M0".format(id)
                   }

            } 
list_pos_min_1 = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).map(lambda x: list(map(lambda x1: int(x1,2),x))).first()
#list(gendata(list_pos_min_1,1))
helpers.bulk(es, gendata(list_pos_min_1,1),request_timeout=1000)

(68, [])

## MultiSearch 

In [46]:
list_pos_min_1 = [vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,8)).map(lambda x: list(map(lambda x1: int(x1,2),x))).first()]
min_1 = sc.parallelize(list_pos_min_1)

In [65]:
def gendata1(list_pos_in_hash):
    for i,pos in enumerate(list_pos_in_hash):
        if(pos != 0):
            yield {"index": "hash{}".format(i) }
            yield {
                    "query": {"terms": {"_id":[str(pos)]}}} 


In [88]:
def realize_search(data_consult):
    from elasticsearch import Elasticsearch
    es = Elasticsearch()
    data_response = es.msearch(body=data_consult)
    return list(map(lambda x : x['hits']['hits'][0]['_source']['text'], data_response['responses']))
    
    

In [92]:
list_ = min_1.map(lambda x : list(gendata1(x))).map(lambda x: realize_search(x)).collect()
list_

[['P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0',
  'P1_M0']]

In [93]:
#list_ = [["P2_M1 P1_M4 ","P2_M1 P3_M2 P3_M2 P1_M4","P2_M1 P2_M3 P1_M4","P2_M1 P4_M1 P1_M4"],["P2_M4 P1_M4","P2_M1 P3_M2 P3_M2","P2_M1 P2_M3","P2_M1 P4_M1 P1_M1"]]

words_list = sc.parallelize(list_)

In [94]:

import numpy as np

def func_count_byKey_return_max(x):
    x_,y_ = np.unique(x,return_counts=True)
    y_max = np.max(y_)
    return " ".join([ i[0].split('_')[0] for i in list(filter(lambda x: x[1] == y_max,list(map(lambda x,y:(x,y) ,x_,y_)))) ])

def func_rank_elements(persons):
    persons_list, count_ = np.unique(persons.split(" "),return_counts=True)
    return persons_list[np.argsort(-1*count_)]


In [95]:
%time
persons = words_list.map(lambda x : " ".join(x)).map(lambda x : x.split()).map(func_count_byKey_return_max ).reduce(lambda x,y: y + " "+ x)
func_rank_elements(persons)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 13.6 µs


array(['P1'], dtype='<U2')